# Env Preparation

## Install Dependencies

In [ ]:
%pip install -r requirements.txt

## Show GPU Information

In [2]:
!nvidia-smi

Mon Feb 24 19:52:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77.01              Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080 ...    On  |   00000000:01:00.0  On |                  N/A |
|  0%   33C    P8             13W /  320W |    2283MiB /  16376MiB |     21%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available: {torch.cuda.is_available()}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

    # Iterate through each available GPU
    for i in range(torch.cuda.device_count()):
        print(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  - CUDA Capability: {torch.cuda.get_device_capability(i)}")
        print(f"  - Memory Allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
        print(f"  - Memory Cached: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
else:
    print("CUDA is not available.")


CUDA is available: True
Number of GPUs: 1

GPU 0: NVIDIA GeForce RTX 4080 SUPER
  - CUDA Capability: (8, 9)
  - Memory Allocated: 0.00 MB
  - Memory Cached: 0.00 MB
  - Current Temperature: Not accessible via PyTorch


# Data Loading, Preprocessing, Tokenization, & Embedding

## Load Data from `datasets`

In [4]:
from datasets import load_dataset

# Load the WMT14 dataset (English-German)
dataset = load_dataset("wmt14", "de-en")

# Preview
print(dataset["train"][0])


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


In [5]:
## Toeknize the Dataset with Bert

In [6]:
from transformers import AutoTokenizer

# Load BERT tokenizer (or any model-specific tokenizer)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset (batch-wise)
def tokenize_function(example):
    example_translations = example["translation"]
    src_corpus = list(map(lambda x: x['en'], example_translations))
    tgt_corpus = list(map(lambda x: x['de'], example_translations))
    src_tokens = tokenizer(src_corpus, padding="max_length", truncation=True, max_length=128)
    tgt_tokens = tokenizer(tgt_corpus, padding="max_length", truncation=True, max_length=128)
    return {
        "src_input_ids": src_tokens["input_ids"],
        "src_attn_masks": src_tokens["attention_mask"],
        "tgt_input_ids": tgt_tokens["input_ids"],
        "tgt_attn_masks": tgt_tokens["attention_mask"]
    }

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["translation"])

# Check result
print(tokenized_dataset["train"][0])


Map:   0%|          | 0/3003 [00:00<?, ? examples/s]

{'src_input_ids': [101, 24501, 24237, 3508, 1997, 1996, 5219, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'src_attn_masks': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tgt_input_ids': [101, 15536, 14728, 2527, 16093, 15272, 4168, 4315, 4133, 9759, 3070, 17668, 3695, 3207, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Create the Dataloader

In [22]:
from torch.utils.data import DataLoader

# Convert dataset to PyTorch format
tokenized_dataset.set_format(type="torch", columns=["src_input_ids", "src_attn_masks", "tgt_input_ids", "tgt_attn_masks"])

# Create DataLoader
train_loader = DataLoader(
    tokenized_dataset["train"], 
    batch_size=16, 
    shuffle=True, 
    num_workers=4, 
    pin_memory=True,  # Faster transfer to GPU
    prefetch_factor=4,  # Preload batches in advance
)

# Iterate through batches
for batch in train_loader:
    print(batch["src_input_ids"].shape)
    break


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

torch.Size([16, 128])


## Release Memory from Redundant Variables

In [8]:
del dataset

# Load and Train the Model

## Load the Transformer Model

In [9]:
from model.transformer import Transformer

# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
vocab_size = tokenizer.vocab_size
model = Transformer(vocab_size, 128).to(device)

# Count total parameters in the Transformer model
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"\n🔍 Model Parameter Summary:")
    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Non-Trainable Parameters: {non_trainable_params:,}\n")

# Call the function
count_parameters(model)

Using device: cuda

🔍 Model Parameter Summary:
Total Parameters: 60,190,010
Trainable Parameters: 60,190,010
Non-Trainable Parameters: 0



## Loss and Optimizer

In [10]:
import torch.nn as nn
import torch.optim as optim

# Loss and Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

## Target Masks

In [11]:
def generate_combined_target_mask(attention_mask):
    """
    Generate combined target mask for decoder:
    1. Padding mask from tokenizer
    2. Lower triangular causal mask

    Args:
        attention_mask (Tensor): [batch_size, tgt_seq_length]

    Returns:
        Tensor: Combined mask [batch_size, tgt_seq_length, tgt_seq_length]
    """
    batch_size, seq_len = attention_mask.shape

    # 1. Generate causal mask (lower triangular)
    causal_mask = torch.tril(torch.ones((seq_len, seq_len), dtype=torch.bool, device=attention_mask.device))
    causal_mask = causal_mask.unsqueeze(0).expand(batch_size, -1, -1)  # Shape: [batch_size, seq_len, seq_len]

    # 2. Expand padding mask for broadcasting
    padding_mask = attention_mask.unsqueeze(1).expand(-1, seq_len, -1)  # [batch_size, seq_len, seq_len]

    # 3. Combine both masks (AND operation)
    combined_mask = causal_mask & padding_mask

    return combined_mask

## Load BERT Model for Embedding

In [12]:
from transformers import BertModel

bert = BertModel.from_pretrained("bert-base-uncased").to(device)

## Training

In [ ]:
from tqdm import tqdm

num_epochs = 1

for epoch in range(num_epochs):
    model.train()  # Set to training mode
    total_loss = 0

    for batch in tqdm(train_loader):
        input_ids = batch["src_input_ids"].to(device)
        attention_mask = batch["src_attn_masks"].to(device)
        labels = batch["tgt_input_ids"].to(device)
        label_mask = batch["tgt_attn_masks"].to(device)
        causal_mask = generate_combined_target_mask(label_mask).to(device)

        # Embed inputs and outputs using BERT (Batch-wise)
        with torch.no_grad():
            embedded_inputs = bert(input_ids, attention_mask=attention_mask).last_hidden_state
            embedded_outputs = bert(labels, attention_mask=label_mask).last_hidden_state

        # Forward pass through Transformer
        outputs = model(
            embedded_inputs, 
            embedded_outputs, 
            attention_mask.unsqueeze(1).unsqueeze(1), 
            causal_mask.unsqueeze(1),
        )  # Customize tgt_masks as needed

        # Compute loss (ignore padding tokens)
        logits = outputs.view(-1, vocab_size)
        labels = labels.view(-1)
        loss = criterion(logits, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Log Epoch Progress
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f}")

# Save Model
torch.save(model.state_dict(), "transformer_model.pth")
print("✅ Training complete and model saved!")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...                                                                                | 0/281800 [00:00<?, ?it/s]
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	-

KeyboardInterrupt: 